# Llama-3-8B 效能基準測試 (vLLM vs TGI Simulation)

本筆記本用於在 Google Colab (T4 GPU) 上測試 vLLM 的實際推論速度。
由於 Colab 免費版的資源限制，這裡主要演示 **vLLM** 的設定與效能測量。

**注意**：Llama-3 模型需要 Hugging Face 存取權限，請確保您已在 Hugging Face 申請通過用且擁有 Token。

In [ ]:
# 1. 安裝必要套件
!pip install vllm pandas huggingface_hub

In [ ]:
# 2. 登入 Hugging Face (貼上您的 HF Token)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 3. 準備測試數據
import time
from vllm import LLM, SamplingParams

# 測試 prompts
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "Python is a programming language that",
] * 4  # 重複幾次以增加負載

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=256)

In [ ]:
# 4. 初始化 vLLM 引擎 (使用 Llama-3-8B)
# 注意：Colab T4 記憶體有限，我們使用 gpu_memory_utilization 控制顯存
llm = LLM(model="meta-llama/Meta-Llama-3-8B", 
          dtype="float16", 
          gpu_memory_utilization=0.9,
          max_model_len=4096)

print("vLLM Engine initialized!")

In [ ]:
# 5. 執行基準測試
start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

end_time = time.time()
total_duration = end_time - start_time

# 計算統計數據
total_tokens = sum([len(output.outputs[0].token_ids) for output in outputs])
throughput = total_tokens / total_duration

print(f"\n=== Benchmark Results ===")
print(f"Total Requests: {len(prompts)}")
print(f"Total Tokens Generated: {total_tokens}")
print(f"Total Time: {total_duration:.2f} s")
print(f"Throughput (TPS): {throughput:.2f} tokens/s")